In [ ]:
# Step 1 — Import Required Libraries
import tensorflow as tf
import tensorflow_datasets as tfds          # for loading tf_flowers dataset
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [ ]:
# 
# Step 2 — Load and Preprocess Dataset (tf_flowers)
# 

# Load the TensorFlow Flowers dataset (70% for training, 30% for testing)
(train_ds, test_ds) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"],
    as_supervised=True  # returns (image, label)
)

# Define constants
IMG_SIZE = (150, 150)   # Resize all images to 150x150 pixels
NUM_CLASSES = 5         # There are 5 flower categories in the dataset

# Function to preprocess images and labels
def preprocess(img, label):
    img = tf.image.resize(img, IMG_SIZE)           # Resize all images
    img = preprocess_input(img)                    # Normalize images (for VGG16)
    label = tf.one_hot(label, NUM_CLASSES)         # Convert labels to one-hot encoding
    return img, label

# Apply preprocessing, batching, and prefetching for faster loading
train_ds = train_ds.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
# 
# Step 3 — Load Pre-trained Model (VGG16)
# 

# Load VGG16 model trained on ImageNet, without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze base model layers (to keep pre-trained features fixed)
base_model.trainable = False

# Build a new model by adding custom classification layers on top
model = models.Sequential([
    base_model,                              # Pre-trained convolutional base
    layers.Flatten(),                        # Flatten output for dense layers
    layers.Dense(64, activation='relu'),     # Custom dense layer
    layers.Dense(NUM_CLASSES, activation='softmax')  # Output layer (5 flower classes)
])

In [ ]:
# 
# Step 4 — Compile and Train Model (Initial Training)
# 

# Compile model with optimizer, loss, and evaluation metrics
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train model for 3 epochs (frozen base)
history = model.fit(train_ds, validation_data=test_ds, epochs=3)

In [ ]:
# 
# Step 5 — Fine-Tune Model (Unfreeze Top Layers)
# 

# Unfreeze last 4 layers of VGG16 for fine-tuning
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Recompile with a lower learning rate to prevent large updates
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train again for 2 epochs with fine-tuning enabled
fine_history = model.fit(train_ds, validation_data=test_ds, epochs=2)

# 
# Step 6 — Evaluate the Final Model
# 

loss, acc = model.evaluate(test_ds)
print("Final Accuracy:", acc)


In [ ]:
# 
# Step 7 — Plot Accuracy Graph
# 

# Combine both training phases for visualization
plt.plot(history.history['accuracy'] + fine_history.history['accuracy'])
plt.title('Training Accuracy (with Fine-tuning)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()


In [ ]:
# #| Question                             | Answer                                                                                             |
# | ------------------------------------ | -------------------------------------------------------------------------------------------------- |
# | What is transfer learning?           | Using a pre-trained model (like VGG16) trained on a large dataset to solve a new, smaller problem. |
# | Why use VGG16?                       | It is a well-known CNN architecture trained on ImageNet; great for feature extraction.             |
# | What does “include_top=False” mean?  | It removes the original classification layers so we can add our own.                               |
# | Why freeze the convolutional layers? | To retain previously learned features and prevent retraining from scratch.                         |
# | What is fine-tuning?                 | Unfreezing a few top layers of a pre-trained model to improve performance on new data.             |
# | What optimizer is used?              | Adam optimizer (adaptive learning rate).                                                           |
# | What is the loss function?           | Categorical crossentropy, suitable for multi-class classification.                                 |
# | How many output classes are there?   | 5 (since the dataset `tf_flowers` has 5 flower types).                                             |
# | What is the image size used?         | All images resized to 150×150 pixels.                                                              |
# | What is one-hot encoding?            | It converts categorical labels (0–4) into binary vectors like [1,0,0,0,0].                         |

# #